In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 22.6 MB/s 
     |████████████████████████████████| 182 kB 59.1 MB/s 
     |████████████████████████████████| 7.6 MB 47.1 MB/s 


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 40.6 MB/s 


In [ ]:
from transformers import AutoModel
from transformers import TFAutoModel, AutoTokenizer
#checkpoint = "drive/MyDrive/archive"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = TFAutoModel.from_pretrained('microsoft/deberta-v3-small', output_hidden_states = True)
from transformers import DebertaTokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/565M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaV2Model.

All the layers of TFDebertaV2Model were initialized from the model checkpoint at microsoft/deberta-v3-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2Model for predictions without further training.


In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/')

('/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/spm.model',
 '/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/nlp_saved_models/deberta-v3small-tokenizer/tokenizer.json')

In [ ]:
def deberta_encode(texts, tokenizer=tokenizer):
    input_ids = []
    # token_type_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, max_length=512, truncation=True, padding='max_length',
                         add_special_tokens=True)
        input_ids.append(token['input_ids'])
        # token_type_ids.append(token['token_type_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(attention_mask)

def get_dataset(df):
    inputs = deberta_encode(df['full_text'])
    targets = np.array(df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']], dtype="float32")
    return inputs, targets

In [ ]:
import re 
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_df = pd.read_csv('/content/sample_data/dataset.csv')
test_df = pd.read_csv('/content/sample_data/test.csv')

In [ ]:
train_df['full_text'] = train_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)
test_df['full_text'] = test_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)

In [ ]:
train_df['num_words'] = train_df['full_text'].apply(lambda x: len(x.split()))
avg_words = round(train_df['num_words'].mean())
max_words = round(train_df['num_words'].max())
print('Average length: {}'.format(avg_words))
print('Max length: {}'.format(max_words))

Average length: 426
Max length: 1250


In [ ]:
train_dataset = get_dataset(train_df)

In [ ]:
def get_tf_model():
    input_ids = tf.keras.layers.Input(
        shape=(512,), dtype=tf.int32, name="input_ids"
    )
    
    attention_masks = tf.keras.layers.Input(
        shape=(512,), dtype=tf.int32, name="attention_masks"
    )
   
    deberta_output = model.deberta(
        input_ids, attention_mask=attention_masks
    )
    
    last_hidden_statee = deberta_output.last_hidden_state
    
    x = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_statee)
    x = tf.keras.layers.Dense(6, activation='sigmoid')(x)
    output = tf.keras.layers.Rescaling(scale=4.0, offset=1.0)(x)
    tfmodel = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)
    tfmodel.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
        loss = tf.keras.losses.mean_squared_error
    )
    return tfmodel

In [ ]:
tfmodel = get_tf_model()

In [ ]:
tfhistory = tfmodel.fit(x=train_dataset[0], y=train_dataset[1], validation_split=0.2, epochs=10, batch_size=8)

Epoch 1/10


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


391/391 [==============================] - 343s 846ms/step - loss: 0.2991 - val_loss: 0.2577
Epoch 2/10
391/391 [==============================] - 330s 843ms/step - loss: 0.2019 - val_loss: 0.2265
Epoch 3/10
391/391 [==============================] - 330s 843ms/step - loss: 0.1529 - val_loss: 0.2495
Epoch 4/10
391/391 [==============================] - 329s 842ms/step - loss: 0.1184 - val_loss: 0.2415
Epoch 5/10
391/391 [==============================] - 329s 842ms/step - loss: 0.0995 - val_loss: 0.2373
Epoch 6/10
391/391 [==============================] - 329s 841ms/step - loss: 0.0801 - val_loss: 0.2371
Epoch 7/10
391/391 [==============================] - 330s 844ms/step - loss: 0.0627 - val_loss: 0.2618
Epoch 8/10
391/391 [==============================] - 329s 842ms/step - loss: 0.0510 - val_loss: 0.2471
Epoch 9/10
391/391 [==============================] - 329s 841ms/step - loss: 0.0399 - val_loss: 0.2735
Epoch 10/10
391/391 [==============================] - 329s 843ms/step - lo

In [ ]:
tfmodel.save('/content/drive/MyDrive/nlp_saved_models/deberta-v3small-essay.hd5')
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/nlp_saved_models/deberta-v3small-essay.hd5')

In [ ]:
test_df

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [ ]:
text_ids,text_masks = deberta_encode(test_df['full_text'])

In [ ]:
pred = saved_model.predict([text_ids,text_masks])

tf_submission = pd.concat([test_df[['text_id']], pd.DataFrame(pred, columns=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'])], axis=1)
tf_submission.head()
tf_submission.to_csv('submission.csv', index=False)

1/1 [==============================] - 2s 2s/step


In [ ]:
str1 = "It is somewhat rash to assume that the only role of science is to answer the question 'How?'. That was true in the days of Newton, when an educated person could have a grasp in outline of all human knowledge. Science then filled some of the gaps left by the deliberations of the philosopher and the theologian. Since then, it has far outstripped the contributions of both. Philosophy has degenerated into historical study, and has no modern contribution to make. Theology has made no advance since the Middle Ages. The mantle of seeking answers to man's most fundamental questions has fallen on science. Whether these questions will ever be answered is an entirely different matter, but there is no other way ahead. So the topic-statement is fundamentally wrong."

test_ids,test_masks = deberta_encode([str1])

pred = saved_model.predict([test_ids,test_masks])

1/1 [==============================] - 0s 73ms/step


In [ ]:
pred

array([[3.2294986, 3.734465 , 4.016418 , 4.0536194, 3.747434 , 3.9779947]],
      dtype=float32)

In [ ]:
lst=[]
for x in pred[0]:
  lst.append(round(x*2)/2.0)

lst

[3.0, 3.5, 4.0, 4.0, 3.5, 4.0]